In [1]:
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU, MaxPooling2D
from keras.regularizers import l2, activity_l2
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Activation, Reshape
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import scipy.misc
import random
import numpy as np
import time
import cv2
import pickle
import os

Using TensorFlow backend.


In [2]:
delta_driving_log=pd.read_csv('./data/deltatrainer_driving_log.csv').sample(frac=1.0)

delta_driving_log.describe()

,steering,throttle,brake,speed
count,840.000000,840.000000,840.0,840.000000
mean,0.037240,0.744158,0.0,26.596302
std,0.217214,0.412747,0.0,7.672159
min,-0.942695,0.000000,0.0,1.121875
25%,-0.031274,0.687741,0.0,30.108208
50%,0.023486,0.985533,0.0,30.183395
75%,0.167014,0.985533,0.0,30.186260
max,1.000000,0.985533,0.0,30.196270


In [3]:
f = open('./gallery.p', 'rb')   
gallery = pickle.load(f)      
f.close() 
print(len(gallery))

24108


In [4]:
def augment_brightness(image):
    image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    
    image[:,:,2] = image[:,:,2]*random_bright
    return image

def trans_image(image,steer,trans_range=50, trans_y=False):
    """
    translate image and compensate for the translation on the steering angle
    """
    assert(image is not None)
    rows, cols, chan = image.shape
    
    # horizontal translation with 0.008 steering compensation per pixel
    tr_x = trans_range*np.random.uniform()-trans_range/2
    steer_ang = steer + tr_x/trans_range*.4
    
    # option to disable vertical translation (vertical translation not necessary)
    if trans_y:
        tr_y = 40*np.random.uniform()-40/2
    else:
        tr_y = 0
    
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
    image_tr = cv2.warpAffine(image,Trans_M,(cols,rows))
    
    return image_tr,steer_ang

pts1 = np.float32([[140,60],[180,60],[0,100],[320,100]])
pts2 = np.float32([[140,0],[180,0],[0,120],[320,120]])

M = cv2.getPerspectiveTransform(pts1,pts2)

def transform(img):
    dst = cv2.warpPerspective(img,M,(320,160))
    dst = cv2.resize(dst, (64,64))
    return dst / 255 - 0.5

In [5]:
y_train_log = []
def generate_batch_samples(df, batch_size=128):    
    camera_shift_rate = {0:0, 1:0.27, 2:-0.27}
    while 1:
        batch_x, batch_y = [], []
        for i in range(batch_size):
            row = random.choice(df)
            # randomly pick between left right and center camaras
            rnd = random.randint(0,2)
            
            image = gallery[row[rnd].strip()]
            
            # compensate -0.2 for right camera and 0.2 for left camera 
            angle = row[3] + camera_shift_rate[rnd] 
            angle = max(-1, angle)
            angle = min(1, angle)
            
            # horizontally random shift the image
            image, angle = trans_image(image, angle)
            
            # brightness augmentation
            if random.random() >0.5:
                image = augment_brightness(image)
                
            #randomly flip
            if random.random() >0.5:
                image = cv2.flip(image, 0)
                angle = -angle
            
            batch_x.append(transform(image))
            batch_y.append(angle)
            y_train_log.append(angle)
        yield shuffle(np.array(batch_x), np.array(batch_y))
                
    
y_valid_log = []
def generate_batch_valid(df, batch_size=128):
    while 1:
        batch_x, batch_y = [], []
        for i in range(batch_size):
            row = random.choice(df)
            image = gallery[row[0].strip()]
            angle = row[3]
            
            batch_x.append(transform(image))
            batch_y.append(angle)
            y_valid_log.append(angle)
        yield shuffle(np.array(batch_x), np.array(batch_y))
            

In [6]:
def getVGG16model(input_shape=(64,64,3)):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='elu')(x)
    x = Dropout(0.6)(x)
    x = Dense(128, activation='elu')(x)
    x = Dropout(0.7)(x)
    prediction = Dense(1, name='prediction')(x)

    model = Model(input=base_model.input, output=prediction)

    optimizer = Adam()
    model.compile(loss='mse', optimizer=optimizer)
    
    return model

In [7]:
def get_commaai_model(input_shape=(64,64,3)):
    model = Sequential()
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same",input_shape=input_shape))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse")

    return model

In [8]:
def nvidia_model(input_shape=(64,64,3)):
    INIT='glorot_uniform' # 'he_normal', glorot_uniform
    keep_prob = 0.2
    reg_val = 0.01
    
    model = Sequential()
    model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode="valid", init=INIT,input_shape=input_shape, W_regularizer=l2(reg_val)))
    # W_regularizer=l2(reg_val)
    model.add(ELU())
    model.add(Dropout(keep_prob))

    model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode="valid", init=INIT))
    model.add(ELU())
    model.add(Dropout(keep_prob))
    
    model.add(Convolution2D(48, 5, 5, subsample=(2, 2), border_mode="valid", init=INIT))
    model.add(ELU())
    model.add(Dropout(keep_prob))

    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid", init=INIT))
    model.add(ELU())
    model.add(Dropout(keep_prob))

    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid", init=INIT))
    model.add(ELU())
    model.add(Dropout(keep_prob))
    
    model.add(Flatten())

    model.add(Dense(100))
    model.add(ELU())
    model.add(Dropout(0.2))
    
    model.add(Dense(50))
    model.add(ELU())
    model.add(Dropout(0.2))
    
    model.add(Dense(10))
    model.add(ELU())
    
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse") # , metrics=['accuracy']
    
    return model


In [9]:
from keras.callbacks import EarlyStopping
epochs = 20
#batch_size = 450

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [10]:
from keras.models import load_model
os.rename('model.h5', 'model.original.h5')
model = load_model('model.original.h5')


In [ ]:
kf = KFold(3, shuffle=True)
for train_index, test_index in kf.split(delta_driving_log):
    batch_size = len(test_index)
    df_train, df_valid = delta_driving_log.as_matrix()[train_index], delta_driving_log.as_matrix()[test_index]
    train_gen = generate_batch_samples(df_train, batch_size=batch_size)
    valid_gen = generate_batch_valid(df_valid, batch_size=batch_size)

    
    # train model with generator
    model.fit_generator(
        train_gen,
        samples_per_epoch=batch_size*22, nb_epoch=epochs,
        validation_data=valid_gen,
        nb_val_samples=batch_size,
        callbacks=[early_stopping]
    )

Epoch 1/20
6160/6160 [==============================] - 11s - loss: 0.1741 - val_loss: 0.0240

In [ ]:
model.save('model.h5')

In [ ]:
### performance review

In [ ]:
from keras.models import load_model
model = load_model('model.h5')

In [ ]:
import cv2
import matplotlib.pyplot as plt

pts1 = np.float32([[140,60],[180,60],[0,100],[320,100]])
pts2 = np.float32([[140,0],[180,0],[0,120],[320,120]])

M = cv2.getPerspectiveTransform(pts1,pts2)

def transform(img):
    dst = cv2.warpPerspective(img,M,(320,160))
    dst = cv2.resize(dst, (64,64))
    return dst

X_test = ['./data/IMG/center_2016_12_01_13_45_23_127.jpg'
          ,'./data/IMG/center_2016_12_01_13_46_37_916.jpg'
          ,'./data/IMG/center_2016_12_01_13_46_27_362.jpg']
Y_test = [-0.5249807, 0, 0.3009732]
for x, y in zip(X_test,Y_test):
    img = scipy.misc.imread(x)
    image_array = transform(np.asarray(img)) / 255 -0.5
    steering_angle = float(model.predict(image_array[None, :, :, :], batch_size=1))
    print('real steering angle {}, predicted {}'.format(y, steering_angle))
    plt.imshow(img)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def inspectHistogram():
    pltChangeSize()
    
    plt.hist(y_train_log, bins=177, label='total')
    plt.hist(y_valid_log, bins=177, label='total')

    plt.title('Image classes Histogram')  
    plt.xlabel('steering angles')  
    plt.ylabel('Counts')  
    plt.legend()  

def pltChangeSize():
    fig_size = plt.rcParams["figure.figsize"]
    # Set figure width to 12 and height to 9
    fig_size[0] = 12
    fig_size[1] = 9
    plt.rcParams["figure.figsize"] = fig_size

inspectHistogram()